In [1]:
import pandas as pd
import os
import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import style
# import seaborn as sns
# import pylab
import random 
from pandas import DataFrame

# import statsmodels.api as sm
import re
from glob import glob

# import matplotlib.pyplot as plt
import math
from statistics import mode
from collections import Counter
from decimal import getcontext
from datetime import date
from datetime import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timedelta
from pandas.tseries.offsets import BDay
import getpass

In [2]:
filepath = "C:/Users/janew/Desktop/DA Project/Quantium Project/Task 1/"
transactionData = pd.read_excel(filepath+"QVI_transaction_data.xlsx")
customerData = pd.read_csv(filepath+"QVI_purchase_behaviour.csv")

In [3]:
'''
Examine Transaction Data, including the following:
    1. change DATE format from int64 to datetime format
    2. perform basic text analysis: remove special characters
    3. examine product names: remove products that are not chips, eg. salsa
    4. examine product quantity: remove those who purchased quantity greater than 200 in one transaction
    5. check missing dates: 2018-12-25 is missing due to Christmas shops closure
    6. check pack size or weight in each product, all products have reasonable pack sizes

Perform summary statistics table

'''

'\nExamine Transaction Data, including the following:\n    1. change DATE format from int64 to datetime format\n    2. perform basic text analysis: remove special characters\n    3. examine product names: remove products that are not chips, eg. salsa\n    4. examine product quantity: remove those who purchased quantity greater than 200 in one transaction\n    5. check missing dates: 2018-12-25 is missing due to Christmas shops closure\n    6. check pack size or weight in each product, all products have reasonable pack sizes\n\nPerform summary statistics table\n\n'

In [4]:
# convert DATE format from int64 to datetime format
def formatDateInCsv(dataframe):
    dataframe['DATE'] = pd.to_datetime(dataframe['DATE'], unit='D', origin='1899-12-30', errors='coerce')

formatDateInCsv(transactionData)
# transactionData['DATE'] = pd.to_datetime(transactionData['DATE'], unit='D', origin='1899-12-30', errors='coerce')
# transactionData

In [5]:
transactionData

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
...,...,...,...,...,...,...,...,...
264831,2019-03-09,272,272319,270088,89,Kettle Sweet Chilli And Sour Cream 175g,2,10.8
264832,2018-08-13,272,272358,270154,74,Tostitos Splash Of Lime 175g,1,4.4
264833,2018-11-06,272,272379,270187,51,Doritos Mexicana 170g,2,8.8
264834,2018-12-27,272,272379,270188,42,Doritos Corn Chip Mexican Jalapeno 150g,2,7.8


In [ ]:
# perform basic text analysis: 
# remove special characters 
transactionData['PROD_NAME'] = transactionData['PROD_NAME'].map(lambda x: re.sub(r'\W+', '', x))

In [ ]:
# remove products that are not chips
Prod_name_pt = transactionData.groupby(['PROD_NAME'])['DATE'].count().reset_index()
Prod_name_pt.rename(columns={'DATE': 'COUNT'}, inplace=True)
Prod_name_pt_clean = transactionData[~transactionData['PROD_NAME'].str.contains('Salsa')]

In [ ]:
# There are no nulls in the columns but product quantity appears to have an outlier which we should investigate
# further. Let’s investigate further the case where 200 packets of chips are bought in one transaction.
# remove outliers: production quantity greater than 200. 
transactionData_clean = Prod_name_pt_clean[Prod_name_pt_clean['PROD_QTY']<200]

In [ ]:
# check if there is any missing dates
transactionData_clean['DATE'].nunique()
# totally 364 different dates, one date is missing
set(transactionData_clean['DATE']) 

# find the one missing date which is 2018-12-25, which makes sense because all shops closed on Christmas holiday. 
k = pd.date_range(start="2018-07-01", end="2019-06-30").difference(transactionData_clean.DATE)

In [ ]:
# check if pack size is reasonable by extracting the last a few digits of PROD_NAME
transactionData_clean['PACK_SIZE'] = transactionData_clean["PROD_NAME"].str.extract("(\d*\.?\d+)", expand=True)
transactionData_clean

In [ ]:
# the largest pack size is 380g and the smallest size is 70g, which are reasonable. 
pack_size_pt = transactionData_clean.groupby(['PACK_SIZE'])['DATE'].count().reset_index()
pack_size_pt.rename(columns={'DATE': 'COUNT'}, inplace=True)
pack_size_pt

In [ ]:
# check brand names
brand_ls = ['RED','SNBTS','INFZNS','WW','SMITH','NCC','DORITO','GRAIN',
            'RRD','SUNBITES','INFUZIONS','WOOLWORTHS','SMITHS','NATURAL','DORITOS','GRNWVES']

In [ ]:
# perform summary statistics table
transactionData_clean.describe()

In [ ]:
'''
Examine Customer Data
'''

In [ ]:
customerData

In [ ]:
# check LIFESTAGE column
lifestage_pt = customerData.groupby(['LIFESTAGE'])['LYLTY_CARD_NBR'].count().reset_index()
lifestage_pt.rename(columns={'LYLTY_CARD_NBR': 'COUNT'}, inplace=True)
lifestage_pt

In [ ]:
# check PREMIUM_CUSTOMER column
premium_pt = customerData.groupby(['PREMIUM_CUSTOMER'])['LYLTY_CARD_NBR'].count().reset_index()
premium_pt.rename(columns={'LYLTY_CARD_NBR': 'COUNT'}, inplace=True)
premium_pt

In [ ]:
transactionData

In [ ]:
customerData

In [ ]:
# No issue with LIFESTAGE and PREMIUM_CUSTOMER columns
# Join TRANSACTION and CUSTOMER tables together
# As the number of rows in data is the same as that of transactionData, we can be sure that no duplicates were created.
joint_df = transactionData.merge(customerData, how = 'left')

In [ ]:
# after left join, check if the newly added columns have null values
joint_df.isnull().values.any()

In [ ]:
joint_df

In [ ]:
# generate a joint data set for further analysis
# joint_df.to_csv(filepath+'QVI_data.csv', index=False)

In [ ]:
'''
Data Analysis on customer segments
    1. Define metrics of interest to the client
        ① who spends the most on chips (total sales), 
        describing customers by lifestage and how premium their general purchasing beahaviour is
        ② how many customers are in each segment
        ③ how many chips are bought per customer by segment
        ④ what's the average chip price by customer segment
'''

In [ ]:
# calculate TOTAL SALES by LIFESTAGE and PREMIUM_CUSTOMER 
# describe which customer segment contribute most to chip sales
lifestage_df = joint_df.groupby(['LIFESTAGE','PREMIUM_CUSTOMER'])['TOT_SALES'].sum().reset_index()

In [ ]:
lifestage_df['%'] = round(100 * lifestage_df['TOT_SALES']  / lifestage_df['TOT_SALES'].sum(), 2)

In [ ]:
lifestage_df = lifestage_df.drop(columns = ['TOT_SALES'])
lifestage_df

In [ ]:
lifestage_df.plot.bar(x='LIFESTAGE', stacked=True, title='Total Sales by Customer Segments')
# Sales are coming mainly from Budget - older families, Mainstream - young singles/couples, and Mainstream - retirees

In [ ]:
lifestage_df = joint_df.groupby(['PREMIUM_CUSTOMER','LIFESTAGE'])['TOT_SALES'].sum().reset_index()
lifestage_df['%'] = round(100 * lifestage_df['TOT_SALES']  / lifestage_df['TOT_SALES'].sum(), 2)
# There are more Mainstream - young singles/couples and Mainstream - retirees who buy chips. This contributes
# to there being more sales to these customer segments but this is not a major driver for the Budget
# - Older families segment.C

In [ ]:
lifestage_df

In [ ]:
joint_df

In [ ]:
# Number of units of chips being bought per customer
# note: perform 'count' aggregate function on LYLTY_CARD_NBR, 
# and perform 'sum' aggregate function on PROD_QTY
avg_units_df_cust = joint_df.groupby(['LIFESTAGE','PREMIUM_CUSTOMER'])['LYLTY_CARD_NBR'].count().reset_index()

In [ ]:
avg_units_df_cust

In [ ]:
avg_units_df_qty = joint_df.groupby(['LIFESTAGE','PREMIUM_CUSTOMER'])['PROD_QTY'].sum().reset_index()

In [ ]:
avg_units_df_qty

In [ ]:
# join two sub-df on LIFESTAGE
avg_units_df = avg_units_df_cust.merge(avg_units_df_qty, on = ['LIFESTAGE','PREMIUM_CUSTOMER'])

In [ ]:
avg_units_df

In [ ]:
avg_units_df['avg units/customer'] = round(avg_units_df['PROD_QTY']/avg_units_df['LYLTY_CARD_NBR'], 2)

In [ ]:
avg_units_df.sort_values('LIFESTAGE')
# older families and younger families in general buy more chips per customer

In [ ]:
# Calculate the average chip price by customer segment
avg_price = joint_df.groupby(['LIFESTAGE','PREMIUM_CUSTOMER'])['TOT_SALES','PROD_QTY'].sum().reset_index()

In [ ]:
avg_price['avg price/unit'] = round(avg_price['TOT_SALES']/avg_price['PROD_QTY'], 2)
avg_price.sort_values('avg price/unit', ascending = False)
# Mainstream midage and young singles and couples are more willing to pay more per packet of chips compared to their budget and premium counterparts.

# since the difference is not large, can check if this difference is statistically significant. 

In [ ]:
'''
Deep dive into specific customer segments for insights
Look at Mainstreams-Young/Single Couples customers who contribute the most to sales and retain them
    1. if Mainstream tend to buy a particular brand of chips
'''

In [ ]:
joint_df

In [ ]:
target_lifestage = ['YOUNG SINGLES/COUPLES']
target_prem_cust = ['Mainstream']

target_qty = joint_df[joint_df['LIFESTAGE'].isin(target_lifestage) & joint_df['PREMIUM_CUSTOMER'].isin(target_prem_cust)]
non_target_qty = joint_df[~(joint_df['LIFESTAGE'].isin(target_lifestage) & joint_df['PREMIUM_CUSTOMER'].isin(target_prem_cust))]

In [ ]:
target_qty = target_qty.groupby(['PROD_NAME'])['PROD_QTY'].sum().reset_index()

In [ ]:
target_qty.rename(columns={'PROD_QTY': 'PROD_QTY_target'}, inplace=True)

In [ ]:
target_qty

In [ ]:
non_target_qty = non_target_qty.groupby(['PROD_NAME'])['PROD_QTY'].sum().reset_index()

In [ ]:
non_target_qty.rename(columns={'PROD_QTY': 'PROD_QTY_non_target'}, inplace=True)

In [ ]:
non_target_qty

In [ ]:
qty_df = target_qty.merge(non_target_qty, on = 'PROD_NAME')

In [ ]:
qty_df['Total_Qty'] = qty_df['PROD_QTY_target'] + qty_df['PROD_QTY_non_target']

In [ ]:
qty_df
# We can see that :
# • Mainstream young singles/couples are 23% more likely to purchase Tyrrells chips compared to the rest of the population
# • Mainstream young singles/couples are 56% less likely to purchase Burger Rings compared to the rest of the population